In [1]:
import pandas as pd
import numpy as np

In [2]:
#val_results = pd.read_csv('../doc/models/model_02/model_02.csv')
#val_results = pd.read_csv('../../doc/models/model_02r/model_02r.csv')
val_results = pd.read_csv('../../doc/models/model_02s/model_02s.csv')
val_results.head()

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score
0,sub-100079,ses-V02,1,T2w,sub-100079_ses-V02_run-1_T2w.nii.gz,3.0,NaN
1,sub-100247,ses-V02,1,T2w,sub-100247_ses-V02_run-1_T2w.nii.gz,3.0,NaN
2,sub-100524,ses-V02,1,T2w,sub-100524_ses-V02_run-1_T2w.nii.gz,1.0,-0.12839
3,sub-100586,ses-V02,1,T2w,sub-100586_ses-V02_run-1_T2w.nii.gz,0.0,NaN
4,sub-100642,ses-V02,1,T2w,sub-100642_ses-V02_run-1_T2w.nii.gz,0.0,NaN


In [3]:
# find scans with highest prediction error
val_results['abs_error'] = abs(val_results['QU_motion'] - val_results['predicted_qu_motion_score'])
val_results = val_results.sort_values(by='abs_error', ascending=False)
val_results.head(15)

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score,abs_error
271,sub-125924,ses-V02,1,T2w,sub-125924_ses-V02_run-1_T2w.nii.gz,0.0,2.380719,2.380719
879,sub-179995,ses-V04,1,T2w,sub-179995_ses-V04_run-1_T2w.nii.gz,3.0,0.744885,2.255115
513,sub-145456,ses-V02,1,T2w,sub-145456_ses-V02_run-1_T2w.nii.gz,3.0,0.861875,2.138125
558,sub-148500,ses-V02,3,T2w,sub-148500_ses-V02_run-3_T2w.nii.gz,3.0,0.870894,2.129106
1022,sub-189059,ses-V02,1,T2w,sub-189059_ses-V02_run-1_T2w.nii.gz,3.0,0.899927,2.100073
160,sub-115876,ses-V03,2,T2w,sub-115876_ses-V03_run-2_T2w.nii.gz,0.0,2.066113,2.066113
503,sub-144495,ses-V02,2,T1w,sub-144495_ses-V02_run-2_T1w.nii.gz,3.0,1.007463,1.992537
667,sub-156873,ses-V04,1,T1w,sub-156873_ses-V04_run-1_T1w.nii.gz,0.0,1.883602,1.883602
588,sub-151084,ses-V03,2,T2w,sub-151084_ses-V03_run-2_T2w.nii.gz,3.0,1.146495,1.853505
514,sub-145456,ses-V02,2,T2w,sub-145456_ses-V02_run-2_T2w.nii.gz,3.0,1.147615,1.852385


In [5]:
# save first 15 rows to csv
val_results.head(15).to_csv('../../data/validation_errors/model_02s/validation_set_top15_errors.csv', index=False)

In [6]:
# remove the 5 highest error scans
val_results = val_results.iloc[5:]
val_results.head()

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score,abs_error
160,sub-115876,ses-V03,2,T2w,sub-115876_ses-V03_run-2_T2w.nii.gz,0.0,2.066113,2.066113
503,sub-144495,ses-V02,2,T1w,sub-144495_ses-V02_run-2_T1w.nii.gz,3.0,1.007463,1.992537
667,sub-156873,ses-V04,1,T1w,sub-156873_ses-V04_run-1_T1w.nii.gz,0.0,1.883602,1.883602
588,sub-151084,ses-V03,2,T2w,sub-151084_ses-V03_run-2_T2w.nii.gz,3.0,1.146495,1.853505
514,sub-145456,ses-V02,2,T2w,sub-145456_ses-V02_run-2_T2w.nii.gz,3.0,1.147615,1.852385


In [7]:
# count NaN values in abs_error column
val_results['abs_error'].isna().sum()

np.int64(814)

In [8]:
# drop rows with NaN values in abs_error column
val_results = val_results.dropna(subset=['abs_error'])
val_results['abs_error'].isna().sum()

np.int64(0)

In [9]:
def create_correlation_coefficient(actual_vals, predicted_vals):
    """Calculate Pearson correlation coefficient between actual and predicted values.

    Args:
        actual_vals (list): Actual QU_motion scores
        predicted_vals (list): Predicted QU_motion scores

    Returns:
        float: Pearson correlation coefficient (r value)
    """
    x = np.array(actual_vals)
    y = np.array(predicted_vals)

    correlation_matrix = np.corrcoef(x, y)
    correlation_coefficient = correlation_matrix[0, 1]

    return correlation_coefficient

In [10]:
# recalculate metrics without the 5 highest error scans
corr = create_correlation_coefficient(val_results['QU_motion'], val_results['predicted_qu_motion_score'])
corr

np.float64(0.6454986193569023)